**1. Install the requirment**

In [1]:
# for mxnet
!pip install --upgrade mxnet

# for pytorch
!pip install torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html

!pip install --upgrade gluoncv

!pip install decord

!pip install mxnet-cu101

!apt-get install protobuf-compiler libprotoc-dev

!pip install onnx==1.7 

     |████████████████████████████████| 46.9MB 64kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 154.6MB 76kB/s 
     |████████████████████████████████| 5.1MB 46.8MB/s 
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.6.0+cpu which is incompatible.
  Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
     |████████████████████████████████| 1.3MB 26.8MB/s 
     |████████████████████████████████| 13.6MB 240kB/s 
     |████████████████████████████████| 356.7MB 47kB/s 
Reading package lists... Done
Building dependency tree       
Reading 

**2. Installing mxnet to onnx**

In [2]:
!rm -r incubator-mxnet
!git clone -b v1.x --single-branch https://github.com/apache/incubator-mxnet.git

rm: cannot remove 'incubator-mxnet': No such file or directory
Cloning into 'incubator-mxnet'...
remote: Enumerating objects: 110169, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 110169 (delta 150), reused 57 (delta 36), pack-reused 109933
Receiving objects: 100% (110169/110169), 78.27 MiB | 23.35 MiB/s, done.
Resolving deltas: 100% (75194/75194), done.


In [3]:
%cd /content/incubator-mxnet/python/mxnet/onnx/
!python setup.py install --force

/content/incubator-mxnet/python/mxnet/onnx
running install
running bdist_egg
running egg_info
creating mx2onnx.egg-info
writing mx2onnx.egg-info/PKG-INFO
writing dependency_links to mx2onnx.egg-info/dependency_links.txt
writing requirements to mx2onnx.egg-info/requires.txt
writing top-level names to mx2onnx.egg-info/top_level.txt
writing manifest file 'mx2onnx.egg-info/SOURCES.txt'
writing manifest file 'mx2onnx.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/mx2onnx
copying mx2onnx/__init__.py -> build/lib/mx2onnx
copying mx2onnx/_export_onnx.py -> build/lib/mx2onnx
copying mx2onnx/_export_helper.py -> build/lib/mx2onnx
copying mx2onnx/_export_model.py -> build/lib/mx2onnx
creating build/lib/mx2onnx/_op_translations
copying mx2onnx/_op_translations/__init__.py -> build/lib/mx2onnx/_op_translations
copying mx2onnx/_op_translations/_op_translations_opset13.py -> build/

**3. Importing the required ones**

In [4]:
from __future__ import division

import argparse, time, logging, os, sys, math

import numpy as np
import mxnet as mx
import gluoncv as gcv
# import onnx 
from mxnet import gluon, nd, init, context
from mxnet import autograd as ag
from mxnet.gluon import nn
from mxnet.gluon.data.vision import transforms

from gluoncv.data.transforms import video
from gluoncv.data import VideoClsCustom
from gluoncv.model_zoo import get_model
from gluoncv.utils import makedirs, LRSequential, LRScheduler, split_and_load, TrainingHistory
import mx2onnx
# from mxnet. import onnx as onnx_mxnet
# from mxnet import onnx 

/usr/local/lib/python3.7/dist-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.8.0` and `torch==1.6.0+cpu` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


**4. Mounting google drive and changing the working directory**

In [5]:
# Don't forget to mount the google drive
%cd /content/drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [6]:
print("MxNet: ", mx.__version__)
print("GluonCv: ", gcv.__version__)
# print("Onnx: ", onnx.__version__)

MxNet:  1.8.0
GluonCv:  0.10.3


**5. Loading the model**

In [7]:
net = get_model(name='i3d_resnet50_v1_custom', nclass=64, pretrained_base=False)
net.load_parameters('Model/i3d_resnet50_v1_88.params')
print(net)

100%|██████████| 208483/208483 [00:05<00:00, 40022.20KB/s]


I3D_ResNetV1(
  (first_stage): HybridSequential(
    (0): Conv3D(3 -> 64, kernel_size=(5, 7, 7), stride=(2, 2, 2), padding=(2, 3, 3), bias=False)
    (1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
    (2): Activation(relu)
    (3): MaxPool3D(size=(1, 3, 3), stride=(2, 2, 2), padding=(0, 1, 1), ceil_mode=False, global_pool=False, pool_type=max, layout=NCDHW)
  )
  (pool2): MaxPool3D(size=(2, 1, 1), stride=(2, 1, 1), padding=(0, 0, 0), ceil_mode=False, global_pool=False, pool_type=max, layout=NCDHW)
  (res_layers): HybridSequential(
    (0): HybridSequential(
      (0): Bottleneck(
        (bottleneck): HybridSequential(
          (0): Conv3D(64 -> 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
          (1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
          (2): Activation(relu)
          (3): Conv3D(64 -> 64, kernel_size=(1, 3, 3), stride=(1

**6. It's time let's convert**

In [8]:
net.hybridize()

# create a sample input and run forward once (required for tracing)
from gluoncv.utils.filesystem import try_import_decord
decord = try_import_decord()

video_fname = 'DataSet/test/005_001_001.mp4'
vr = decord.VideoReader(video_fname)
frame_id_list = range(0, 64, 2)
video_data = vr.get_batch(frame_id_list).asnumpy()
clip_input = [video_data[vid, :, :, :] for vid, _ in enumerate(frame_id_list)]

# transform_fn = video.VideoGroupValTransform(size=(224, 224), mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transform_fn = transforms.Compose([
    # Fix the input video frames size as 256×340 and randomly sample the cropping width and height from
    # {256,224,192,168}. After that, resize the cropped regions to 224 × 224.
    video.VideoMultiScaleCrop(size=(224, 224), scale_ratios=[1.0, 0.875, 0.75, 0.66]),
    # Randomly flip the video frames horizontally
    video.VideoRandomHorizontalFlip(),
    # Transpose the video frames from height*width*num_channels to num_channels*height*width
    # and map values from [0, 255] to [0,1]
    video.VideoToTensor(),
    # Normalize the video frames with mean and standard deviation calculated across all images
    video.VideoNormalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

clip_input = transform_fn(clip_input)
clip_input = np.stack(clip_input, axis=0)
clip_input = clip_input.reshape((-1,) + (32, 3, 224, 224))
clip_input = np.transpose(clip_input, (0, 2, 1, 3, 4))
input_shape = clip_input.shape
print(input_shape)
print('Video data is readed and preprocessed.')

(1, 3, 32, 224, 224)
Video data is readed and preprocessed.


In [10]:
# Map
CLASS_MAP = {
    0: "Opaque",
    1: "Red",
    2: "Green",
    3: "Yellow",
    4: "Bright",
    5: "Light-blue",
    6: "Colors",
    7: "Red",
    8: "Women",
    9: "Enemy",
   10: "Son",
   11: "Man",
   12: "Away",
   13: "Drawer",
   14: "Born",
   15: "Learn",
   16: "Call",
   17: "Skimmer",
   18: "Bitter",
   19: "Sweet milk",
   20: "Milk",
   21: "Water",
   22: "Food",
   23: "Argentina",
   24: "Uruguay",
   25: "Country",
   26: "Last name",
   27: "Where",
   28: "Mock",
   29: "Birthday",
   30: "Breakfast",
   31: "Photo",
   32: "Hungry",
   33: "Map",
   34: "Coin",
   35: "Music",
   36: "Ship",
   37: "None",
   38: "Name",
   39: "Patience",
   40: "Perfume",
   41: "Deaf",
   42: "Trap",
   43: "Rice",
   44: "Barbecue",
   45: "Candy",
   46: "Chewing-gum",
   47: "Spaghetti",
   48: "Yogurt",
   49: "Accept",
   50: "Thanks",
   51: "Shut down",
   52: "Appear",
   53: "To land",
   54: "Catch",
   55: "Help",
   56: "Dance",
   57: "Bathe",
   58: "Buy",
   59: "Copy",
   60: "Run",
   61: "Realize",
   62: "Give",
   63: "Find"
}

# Running the prediction
pred = net(nd.array(clip_input))
topK = 5
ind = nd.topk(pred, k=topK)[0].astype('int')
print('The input video clip is classified to be')
for i in range(topK):
    print('\t[%s], with probability %.3f.'%
          (CLASS_MAP[ind[i].asscalar()], nd.softmax(pred)[0][ind[i]].asscalar()))

The input video clip is classified to be
	[Bright], with probability 0.931.
	[Skimmer], with probability 0.061.
	[Chewing-gum], with probability 0.003.
	[Name], with probability 0.002.
	[Trap], with probability 0.001.


In [11]:
# export your model
net.export("Model_Final_6/I3D_Model_64")

**7. Converting to ONNX**

In [12]:
#convert using onnx

onnx_file='Model_Final_6/output.onnx'
params = 'Model_Final_6/I3D_Model-0000.params'
sym='Model_Final_6/I3D_Model-symbol.json'
mx2onnx.export_model(sym, params, [input_shape], np.float32, onnx_file)
print('onnx export done')

onnx export done


**8. Validating the ONNX**

In [14]:
# validate model
# load the ONNX-model
import onnx
model_proto = onnx.load_model(onnx_file)

# check if the converted ONNX-protobuf is valid
onnx.checker.check_graph(model_proto.graph)